In [2]:
%matplotlib inline
import os
import shutil
from glob import glob

import numpy as np
import xarray as xr

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point
import dask

In [3]:
from ncar_jobqueue import NCARCluster
from dask.distributed import Client
try:
    cluster
    client
except:
    cluster = NCARCluster(cores=1, memory="10 GB", processes=1, walltime="2:00:00")
    cluster.scale(18)
    client = Client(cluster) # Connect this local process to remote workers'
client

Client Scheduler: tcp://10.12.205.32:40887 Dashboard: https://jupyterhub.ucar.edu/dav/user/mclong/proxy/8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [4]:
for f in glob('data/cesm-commitment.*'):
    os.remove(f)

In [6]:
path = '/glade/campaign/cesm/collections/LowWarming/CESM-CAM5-BGC-LowWarming/atm/proc/tseries/daily'
varname = 'TREFHT'
cases = [
    '1pt5degC',
    '1pt5degC_OS',
    '2pt0degC',
]

lon, lat = 5.85573, 52.0407254

for case in cases:
    
    files = sorted(glob(f'{path}/{varname}/b.e11.BRCP26C5CNBDRD.f09_g16.{case}.*.nc'))
    member_ids = sorted(list(set([f.split('.')[-6] for f in files])))
    
    ds_list = []    
    for m in member_ids:
        mem_files = sorted(f for f in files if (f.split('.')[-6] == m))
        
        print(case, m, end=': ')
        print([f.split('.')[-2] for f in mem_files])
        
        with dask.config.set(**{'array.slicing.split_large_chunks': True}):
            dsm = xr.open_mfdataset(mem_files, chunks={'time': 1000}, decode_times=True,)
        ds_list.append(dsm)
    
    member_id = xr.DataArray(member_ids, dims=('member_id'), name='member_id')
    
    with dask.config.set(**{'array.slicing.split_large_chunks': True}):
        ds = xr.concat(ds_list, dim=member_id)

    dsp = ds.interp(lat=lat, lon=lon).reset_coords(['lat', 'lon'], drop=True)
    dsp_mon = dsp.resample(time='M').mean()
    
    for member_id in dsp.member_id.values:
        dsp.TREFHT.sel(member_id=member_id).to_dataframe().to_csv(f'data/cesm-commitment.{case}.TREFHT.{member_id}.csv')        

1pt5degC 001: ['20060101-21001231']
1pt5degC 002: ['20060101-21001231']
1pt5degC 003: ['20060101-21001231']
1pt5degC 004: ['20060101-21001231']
1pt5degC 005: ['20060101-21001231']
1pt5degC 006: ['20110101-21001231']
1pt5degC 007: ['20110101-20501231', '20510101-21001231']
1pt5degC 008: ['20060101-21001231']
1pt5degC 009: ['20260101-21001231']
1pt5degC 009c: ['20060101-21001231']
1pt5degC 010: ['20260101-21001231']
1pt5degC 010c: ['20060101-21001231']
1pt5degC 011: ['20060101-21001231']
1pt5degC 011c: ['20060101-20091231', '20100101-20191231', '20200101-20291231', '20300101-20391231', '20400101-20491231', '20500101-20591231', '20600101-20691231', '20700101-20791231', '20800101-20891231', '20900101-20991231', '20900101-21001231']
1pt5degC 012c: ['20060101-21001231']
1pt5degC 013c: ['20060101-21001231']
1pt5degC 014c: ['20060101-21001231']
1pt5degC_OS 001: ['20060101-21001231']
1pt5degC_OS 002: ['20060101-21001231']
1pt5degC_OS 003: ['20060101-21001231']
1pt5degC_OS 004: ['20060101-210012